In [4]:
%%capture
import os

# unzip data
if not os.path.exists('.data/data'):
  !unzip spotify_million_playlist_dataset.zip -d .data/

In [5]:
import glob
import json
import pandas as pd

# process single file into { pid: track_uri } dictionary
def process_file(filename):
    with open(filename, 'r') as file:
        content = json.load(file)
        playlists_data = [{'pid': playlist['pid'], 'track_uri': track['track_uri']}
                          for playlist in content['playlists']
                          for track in playlist['tracks']]
    return playlists_data

In [6]:
file_prefix = ".data/data/*"

# process all files in directory
def playlist_generator():
    for filename in sorted(glob.glob(file_prefix)):
        yield process_file(filename)

In [7]:
playlist_track_df = pd.DataFrame()

# process data into dataframe with columns (pid, track_uri)
for playlists_chunk in playlist_generator():
    chunk_df = pd.DataFrame(playlists_chunk)
    playlist_track_df = pd.concat([playlist_track_df, chunk_df], ignore_index=True)

# Display the DataFrame
playlist_track_df.head()

,pid,track_uri
0,0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI
1,0,spotify:track:6I9VzXrHxO9rA9A5euc8Ak
2,0,spotify:track:0WqIKmW4BTrj3eJFmnCKMv
3,0,spotify:track:1AWQoqb9bSvzTjaLralEkT
4,0,spotify:track:1lzr43nnXAijIGYnCT8M8H


In [8]:
playlist_track_df.shape

(66346428, 2)

In [9]:
import numpy as np

# create global mapping for trackid: track_uri
track_map = playlist_track_df['track_uri'].unique()
np.save('track_map', track_map)

# create local lookup for track_uri: trackid
lookup = {value: index for index, value in enumerate(track_map)}

# replace track_uri with trackid
playlist_track_df['trackid'] = playlist_track_df['track_uri'].apply(lambda x: lookup[x])
playlist_track_df = playlist_track_df.drop(['track_uri'], axis=1)
playlist_track_df.head()

,pid,trackid
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4


In [27]:
import random

# get test pids
random_sample = random.sample(range(1000000), 200000)

# Create a boolean mask for test set
test_mask = playlist_track_df['pid'].isin(random_sample)

# Filter data into train and test sets based on the mask
test = playlist_track_df[test_mask]
train = playlist_track_df[~test_mask]

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

train['pid'] = train.groupby('pid').ngroup()
test['pid'] = test.groupby('pid').ngroup()

/var/folders/t0/r0qybgm95y5342ymf4p7rzg00000gn/T/ipykernel_17514/1673942246.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pid'] = train.groupby('pid').ngroup()
/var/folders/t0/r0qybgm95y5342ymf4p7rzg00000gn/T/ipykernel_17514/1673942246.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pid'] = test.groupby('pid').ngroup()


In [34]:
from sklearn.model_selection import train_test_split

# split test set into seen and unseen
test_X, test_y = train_test_split(test, test_size = 0.2, stratify=test['pid'])

In [35]:
from scipy import sparse

train_csr = sparse.coo_matrix((np.ones(len(train)), (train["pid"], train["trackid"])), shape=(train['pid'].nunique(), len(track_map))).tocsr()
test_X_csr = sparse.coo_matrix((np.ones(len(test_X)), (test_X["pid"], test_X["trackid"])), shape=(test_X['pid'].nunique(), len(track_map))).tocsr()
test_y_csr = sparse.coo_matrix((np.ones(len(test_y)), (test_y["pid"], test_y["trackid"])), shape=(test_y['pid'].nunique(), len(track_map))).tocsr()

In [36]:
from scipy.sparse import save_npz

save_npz("train_user_csr", train_csr)
save_npz("testx_user_csr", test_X_csr)
save_npz("testy_user_csr", test_y_csr)